In [ ]:
import numpy as np, pandas as pd, pickle, glob
from pathlib import Path

ROOT = Path("..").resolve(); INPUT = ROOT/"input"; WORK = ROOT/"working"; SUB = ROOT/"submissions"
SUB.mkdir(exist_ok=True, parents=True)

ID_COL = "id"      # コンペ仕様に合わせて
TARGET = "target"  # 提出列名（要調整）

models = sorted(glob.glob(str(WORK/"model_fold*.pkl")))
assert models, "working/ に保存済みモデルがありません。020_train.ipynb を先に実行してください。"
print(models)

In [ ]:
test = pd.read_csv(INPUT/"test.csv")
X_test = test.drop(columns=[c for c in [TARGET] if c in test.columns])

# 分類/回帰の区別なく .predict を平均（最小実装）
preds = []
for mp in models:
    with open(mp, 'rb') as f:
        m = pickle.load(f)
    preds.append(m.predict(X_test))

import numpy as np
pred = np.mean(np.vstack(preds), axis=0)

sub = pd.DataFrame({
    ID_COL if ID_COL in test.columns else 'id': test[ID_COL] if ID_COL in test.columns else range(len(test)),
    TARGET: pred
})

out_p = SUB/"submission.csv"
sub.to_csv(out_p, index=False)
print("saved ->", out_p)
sub.head()